# MI BCI classification using Random Forests with hyperparameters optimized in randomized and grid search 

Written by: Sabato Santaniello, Fatemeh Delavari [Atena]

Last Edit: 04/14/2024

Version: 1

In [1]:
import pandas as pd             # Pandas is for data analysis and structure manipulation
import numpy as np              # NumPy is for numerical operations
import sklearn                  # Scikit-Learn is for the ML algorithms
import matplotlib               # MatPlotLib is for making plots & figures
import matplotlib.pyplot as plt # PyPlot is a subset of the library for making MATLAB-style plots

import sys                      # Optional: It is used to print out the current version of Python
import warnings                 # Optional: It is meant to ignore warnings
from datetime import date       # Optional: It is used to print out the date of this solution

## Let us setup the font size
plt.rcParams['axes.labelsize']  = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
print('Done!')

## Print out the configuration of the solver and the current date
warnings.filterwarnings("ignore")
print(f"This script was executed on {date.today()} with the following libraries:\n")
print(f"-- Python ver. {sys.version}")
print(f"-- NumPy ver. {np.__version__}")
print(f"-- Pandas ver. {pd.__version__}")
print(f"-- Scikit-Learn ver. {sklearn.__version__}")

Done!
This script was executed on 2024-04-25 with the following libraries:

-- Python ver. 3.12.2 | packaged by Anaconda, Inc. | (main, Feb 27 2024, 17:28:07) [MSC v.1916 64 bit (AMD64)]
-- NumPy ver. 1.26.4
-- Pandas ver. 2.2.1
-- Scikit-Learn ver. 1.3.0


In [2]:
from scipy.io import loadmat
path = r'C:\Users\Atena\OneDrive - University of Connecticut\DriveCbackup\MIBCI_2024\\'
plvm= loadmat(f'{path}plv1mi.mat')
plvtdL = plvm['plvtL']
plvtdR = plvm['plvtR']

In [3]:
plvtdL.shape

(9, 72, 22, 22)

In [4]:
Y = np.concatenate((
    np.zeros((72, 1)),
    np.ones((72, 1))
), axis=0)

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Test how the optimal classifiers perform in cross-validation
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score

scaler = StandardScaler()

In [18]:
## Create the classifiers and search the space of hyperparameters in a pseudo-random way
obj1 = RandomForestClassifier(random_state=81)

# Cast a wide net of parameter combinations. Note that "GridSearchCV" will test all parameter combinations and apply cross-validation to each combination. 
# Instead, "RandomizedSearchCV" will test only "n_iter" randomly selected parameter combinations and apply cross-validation to each selected combination.

param_grid = {
   #  'criterion':['gini', 'entropy'],
    'criterion':['gini'],
    'n_estimators': np.arange(20, 5000, 100),
    'min_samples_leaf': np.arange(1, 50, 5),
    'max_features':['log2']
   #  'max_features':['auto', 'sqrt', 'log2']
}

Y = np.concatenate((
    np.zeros((72, 1)),
    np.ones((72, 1))
), axis=0)

Rsearch_RF = RandomizedSearchCV(obj1, param_grid, cv=5, n_iter=100, scoring='accuracy', refit=True, return_train_score=False, n_jobs=-1,verbose=1)

RFps = {}
acc = np.zeros([9, 1])
sis = np.zeros([9, 22*22])
sim = np.zeros([9, 22*22])

for d in range(1):
   plvL = np.reshape(plvtdL[d, :, :, :], (72, -1))
   plvR = np.reshape(plvtdR[d, :, :, :], (72, -1))
   plvLR = np.concatenate((plvL, plvR), axis=0)
   X = plvLR

   X = scaler.fit_transform(X)

   Rsearch_RF.fit(X, Y)

## Print the parameters associated with the "best accuracy" in cross validation
   print(Rsearch_RF.best_params_)

   RFps[d] = [Rsearch_RF.best_params_]

   rf_clf = Rsearch_RF.best_estimator_

   ## Define the cross-validation. I decide to shuffle before splitting
   split_info = StratifiedKFold(n_splits=5, shuffle=True, random_state=95)

## Initialize the vectors
   Y_test = np.empty([0,1],dtype='int64')
   Y_pred = np.empty([0,1],dtype='int64')
   Y_pred_proba = np.empty([0,2],dtype='float64')

## Cross-validation loop
   for train_index, test_index in split_info.split(X, Y):
    
       ## Make the split
      X_train, X_test = X[train_index], X[test_index]
      Y_train, tmp = Y[train_index], Y[test_index]
   
      Y_test = np.vstack((Y_test, tmp.reshape(-1, 1)))
   
      ## Remember to apply the scaling
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      ## Fit the classifiers
      rf_clf.fit(X_train,Y_train)
        
      ## Compute the preditions
      Y_pred = np.vstack((Y_pred, rf_clf.predict(X_test).reshape(-1,1)))
    
      ## Compute the probabilities associated with each class
      Y_pred_proba = np.vstack((Y_pred_proba, rf_clf.predict_proba(X_test)))

   classifier = Y_pred
   # label = 'Random Forests'

   # Compute feature importance
   feature_importance = rf_clf.feature_importances_
   sorted_idx = np.argsort(feature_importance)[::-1]
   sorted_importance = feature_importance[sorted_idx]

   # Store sorted indices and importance (assuming sis and sim are pre-initialized arrays)
   # sis[d, :] = sorted_idx
   # sim[d, :] = sorted_importance

   accuracy = accuracy_score(Y_test, classifier)
   acc[d] = accuracy

   # acc = accuracy_score(Y_test, classifier)
   print(accuracy)
      # pre = precision_score(Y_test, classifier, average = 'weighted')
      # rec = recall_score(Y_test, classifier, average = 'weighted')
      # f1s = f1_score(Y_test, classifier, average = 'weighted')
      # print(label,': Acc=',acc,'\t Prec=',pre,'\t Recall=',rec,'\t F1 score=',f1s)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


KeyboardInterrupt: 

In [ ]:
## Create the classifiers and search the space of hyperparameters in a pseudo-random way
obj1 = RandomForestClassifier(random_state=81)

# Cast a wide net of parameter combinations. Note that "GridSearchCV" will test all parameter combinations and apply cross-validation to each combination. 
# Instead, "RandomizedSearchCV" will test only "n_iter" randomly selected parameter combinations and apply cross-validation to each selected combination.

param_grid = {
   #  'criterion':['gini', 'entropy'],
    'criterion':['gini'],
    'n_estimators': np.arange(20, 5000, 100),
    'min_samples_leaf': np.arange(1, 50, 5),
    'max_features':['log2']
   #  'max_features':['auto', 'sqrt', 'log2']
}

Y = np.concatenate((
    np.zeros((72, 1)),
    np.ones((72, 1))
), axis=0)

Rsearch_RF = RandomizedSearchCV(obj1, param_grid, cv=5, n_iter=100, scoring='accuracy', refit=True, return_train_score=False, n_jobs=-1,verbose=1)

RFps = {}
acc = np.zeros([9, 1])
sis = np.zeros([9, 22*22])
sim = np.zeros([9, 22*22])

for d in range(1):
   plvL = np.reshape(plvtdL[d, :, :, :], (72, -1))
   plvR = np.reshape(plvtdR[d, :, :, :], (72, -1))
   plvLR = np.concatenate((plvL, plvR), axis=0)
   X = plvLR

   X = scaler.fit_transform(X)

   Rsearch_RF.fit(X, Y)

## Print the parameters associated with the "best accuracy" in cross validation
   print(Rsearch_RF.best_params_)

   RFps[d] = [Rsearch_RF.best_params_]

   rf_clf = Rsearch_RF.best_estimator_

   ## Define the cross-validation. I decide to shuffle before splitting
   split_info = StratifiedKFold(n_splits=5, shuffle=True, random_state=95)

## Initialize the vectors
   Y_test = np.empty([0,1],dtype='int64')
   Y_pred = np.empty([0,1],dtype='int64')
   Y_pred_proba = np.empty([0,2],dtype='float64')

## Cross-validation loop
   for train_index, test_index in split_info.split(X, Y):
    
       ## Make the split
      X_train, X_test = X[train_index], X[test_index]
      Y_train, tmp = Y[train_index], Y[test_index]
   
      Y_test = np.vstack((Y_test, tmp.reshape(-1, 1)))
   
      ## Remember to apply the scaling
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      ## Fit the classifiers
      rf_clf.fit(X_train,Y_train)
        
      ## Compute the preditions
      Y_pred = np.vstack((Y_pred, rf_clf.predict(X_test).reshape(-1,1)))
    
      ## Compute the probabilities associated with each class
      Y_pred_proba = np.vstack((Y_pred_proba, rf_clf.predict_proba(X_test)))

   classifier = Y_pred
   # label = 'Random Forests'

   # Compute feature importance
   feature_importance = rf_clf.feature_importances_
   sorted_idx = np.argsort(feature_importance)[::-1]
   sorted_importance = feature_importance[sorted_idx]

   # Store sorted indices and importance (assuming sis and sim are pre-initialized arrays)
   # sis[d, :] = sorted_idx
   # sim[d, :] = sorted_importance

   accuracy = accuracy_score(Y_test, classifier)
   acc[d] = accuracy

   # acc = accuracy_score(Y_test, classifier)
   print(accuracy)
      # pre = precision_score(Y_test, classifier, average = 'weighted')
      # rec = recall_score(Y_test, classifier, average = 'weighted')
      # f1s = f1_score(Y_test, classifier, average = 'weighted')
      # print(label,': Acc=',acc,'\t Prec=',pre,'\t Recall=',rec,'\t F1 score=',f1s)


In [19]:
# import scipy.io as sio

# lRFps = list(RFps.values())

# # Create a dictionary of the variables
# data = {
#     'RFps': lRFps,
#     'acc': acc,
#     'sis': sis,
#     'sim': sim
# }

# # Save to a .mat file
# sio.savemat('RFp1.mat', data)


In [6]:
import scipy.io as sio

# Load the .mat file
data = sio.loadmat('RFp1.mat')

# Access the variables
RFps = data['RFps']
acc = data['acc']
sis = data['sis']
sim = data['sim']


In [23]:
RFps[0][0]

array([[(array([[4320]]), array([[11]]), array(['log2'], dtype='<U4'), array(['gini'], dtype='<U4'))]],
      dtype=[('n_estimators', 'O'), ('min_samples_leaf', 'O'), ('max_features', 'O'), ('criterion', 'O')])

In [16]:
acc

array([[0.63888889],
       [0.5625    ],
       [0.75694444],
       [0.73611111],
       [0.48611111],
       [0.59027778],
       [0.58333333],
       [0.83333333],
       [0.6875    ]])

In [ ]:
Y = np.concatenate((
    np.zeros((72, 1)),
    np.ones((72, 1))
), axis=0)

In [17]:
acc_p = np.zeros((9, 100))

for d in range(9):
   plvL = np.reshape(plvtdL[d, :, :, :], (72, -1))
   plvR = np.reshape(plvtdR[d, :, :, :], (72, -1))
   plvLR = np.concatenate((plvL, plvR), axis=0)
   X = plvLR

   X = scaler.fit_transform(X)

   Bparams = RFps[d][0]

   rf_clf_P = RandomForestClassifier(**Bparams)
   for rep in range(100):
      shuffledY = np.random.permutation(Y)
      ## Define the cross-validation. I decide to shuffle before splitting
      split_info = StratifiedKFold(n_splits=5, shuffle=True, random_state=95)

   ## Initialize the vectors
      shuffledY_test = np.empty([0,1],dtype='int64')
      Y_pred = np.empty([0,1],dtype='int64')
      Y_pred_proba = np.empty([0,2],dtype='float64')

   ## Cross-validation loop
      for train_index, test_index in split_info.split(X, Y):
      
         ## Make the split
         X_train, X_test = X[train_index], X[test_index]
         shuffledY_train, tmp = shuffledY[train_index], shuffledY[test_index]
      
         shuffledY_test = np.vstack((shuffledY_test, tmp.reshape(-1, 1)))
      
         ## Remember to apply the scaling
         X_train = scaler.fit_transform(X_train)
         X_test = scaler.transform(X_test)

         ## Fit the classifiers
         rf_clf_P.fit(X_train,shuffledY_train)
         
         ## Compute the preditions
         Y_pred = np.vstack((Y_pred, rf_clf_P.predict(X_test).reshape(-1,1)))
         
         ## Compute the probabilities associated with each class
         Y_pred_proba = np.vstack((Y_pred_proba, rf_clf_P.predict_proba(X_test)))

      classifier = Y_pred
      # label = 'Random Forests'

      accuracy = accuracy_score(shuffledY_test, classifier)
      acc_p[d, rep] = accuracy



TypeError: sklearn.ensemble._forest.RandomForestClassifier() argument after ** must be a mapping, not numpy.ndarray

In [ ]:
mAcc1 = np.mean(acc_p, axis=0)
find(mAcc1[d]>acc[d])
for d in range(9):
    length[d] = np.sum(mAcc1[d] > acc[d])
    

In [72]:
# print(Y_pred_proba.shape)
# print(Y_pred)

# print(acc)
# print(accuracy)

print(acc_p)

[[0.59027778 0.46527778 0.50694444 0.54861111 0.45833333 0.40277778
  0.41666667 0.53472222 0.46527778 0.51388889 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.      

In [41]:
# Gsearch_RF = GridSearchCV(obj1, param_grid, cv=5, scoring='accuracy', refit=True, return_train_score=False, n_jobs=-1,verbose=1)

# for d in range(1):
#    plvL = np.reshape(plvtdL[d, :, :, :], (72, -1))
#    plvR = np.reshape(plvtdR[d, :, :, :], (72, -1))
#    plvLR = np.concatenate((plvL, plvR), axis=0)
#    X = plvLR

#    X = scaler.fit_transform(X)

#    Gsearch_RF.fit(X, Y)

# ## Print the parameters associated with the "best accuracy" in cross validation
#    print(Gsearch_RF.best_params_)

#    rf_clf = Gsearch_RF.best_estimator_

#    ## Define the cross-validation. I decide to shuffle before splitting
#    split_info = StratifiedKFold(n_splits=5, shuffle=True, random_state=95)

# ## Initialize the vectors
#    Y_test = np.empty([0,1],dtype='int64')
#    Y_pred = np.empty([0,1],dtype='int64')
#    Y_pred_proba = np.empty([0,2],dtype='float64')

# ## Cross-validation loop
#    for train_index, test_index in split_info.split(X, Y):
    
#        ## Make the split
#       X_train, X_test = X[train_index], X[test_index]
#       Y_train, tmp = Y[train_index], Y[test_index]
   
#       Y_test = np.vstack((Y_test, tmp.reshape(-1, 1)))
   
#       ## Remember to apply the scaling
#       X_train = scaler.fit_transform(X_train)
#       X_test = scaler.transform(X_test)

#       ## Fit the classifiers
#       rf_clf.fit(X_train,Y_train)
        
#       ## Compute the preditions
#       Y_pred = np.vstack((Y_pred, rf_clf.predict(X_test).reshape(-1,1)))
    
#       ## Compute the probabilities associated with each class
#       Y_pred_proba = np.vstack((Y_pred_proba, rf_clf.predict_proba(X_test)))

#    classifier = Y_pred
#    # label = 'Random Forests'

#    acc = accuracy_score(Y_test, classifier)
#    print(acc)
#       # pre = precision_score(Y_test, classifier, average = 'weighted')
#       # rec = recall_score(Y_test, classifier, average = 'weighted')
#       # f1s = f1_score(Y_test, classifier, average = 'weighted')
#       # print(label,': Acc=',acc,'\t Prec=',pre,'\t Recall=',rec,'\t F1 score=',f1s)


In [7]:
acc1p = np.zeros((9, 1))

for d in range(9):
   plvL = np.reshape(plvtdL[d, :, :, :], (72, -1))
   plvR = np.reshape(plvtdR[d, :, :, :], (72, -1))
   plvLR = np.concatenate((plvL, plvR), axis=0)
   XX = plvLR

   X = XX[:, 1:5]

   X = scaler.fit_transform(X)

   Bparams = RFps[d][0]

   rf_clf_P = RandomForestClassifier(**Bparams)

   ## Define the cross-validation. I decide to shuffle before splitting
   split_info = StratifiedKFold(n_splits=5, shuffle=True, random_state=95)

## Initialize the vectors
   Y_test = np.empty([0,1],dtype='int64')
   Y_pred = np.empty([0,1],dtype='int64')
   Y_pred_proba = np.empty([0,2],dtype='float64')

## Cross-validation loop
   for train_index, test_index in split_info.split(X, Y):
    
       ## Make the split
      X_train, X_test = X[train_index], X[test_index]
      Y_train, tmp = Y[train_index], Y[test_index]
   
      Y_test = np.vstack((Y_test, tmp.reshape(-1, 1)))
   
      ## Remember to apply the scaling
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      ## Fit the classifiers
      rf_clf.fit(X_train,Y_train)
        
      ## Compute the preditions
      Y_pred = np.vstack((Y_pred, rf_clf.predict(X_test).reshape(-1,1)))
    
      ## Compute the probabilities associated with each class
      Y_pred_proba = np.vstack((Y_pred_proba, rf_clf.predict_proba(X_test)))

   classifier = Y_pred
   # label = 'Random Forests'

   # Compute feature importance
   feature_importance = rf_clf.feature_importances_

   accuracy = accuracy_score(Y_test, classifier)
   acc1p[d] = accuracy


TypeError: sklearn.ensemble._forest.RandomForestClassifier() argument after ** must be a mapping, not numpy.ndarray